In [1]:
import pandas as pd
import requests
import GetOldTweets3 as got
import time
import datetime
import os
from ratelimit import limits, sleep_and_retry

# Get list of djia tickers
dow = ['mmm', 'axp', 'aapl', 'ba', 'cat', 'cvx', 'csco', 'ko', 'dow', 'xom', 'gs', 'hd', 'ibm',
      'intc', 'jnj', 'jpm', 'mcd', 'mrk', 'msft', 'nke', 'pfe', 'pg', 'trv', 'unh',
      'vz', 'v', 'wmt', 'wba', 'dis', 'rtn', 'utx'] #  
query_dow = [f'${i}' for i in dow]


In [2]:
def get_twitter_info(tweet_df):
    tweet_df["id"] = tweet_df["got_criteria"].apply(lambda x: x.id)
    tweet_df["tweet_text"] = tweet_df["got_criteria"].apply(lambda x: x.text)
    tweet_df["date"] = tweet_df["got_criteria"].apply(lambda x: x.date)
    tweet_df["hashtags"] = tweet_df["got_criteria"].apply(lambda x: x.hashtags)
    tweet_df["username"] = tweet_df["got_criteria"].apply(lambda x: x.username)
    tweet_df["mentions"] = tweet_df["got_criteria"].apply(lambda x: x.mentions)
    tweet_df["rewtweets"] = tweet_df["got_criteria"].apply(lambda x: x.retweets)
    tweet_df["replies"] = tweet_df["got_criteria"].apply(lambda x: x.replies)
    tweet_df["link"] = tweet_df["got_criteria"].apply(lambda x: x.permalink)
    return tweet_df
# https://medium.com/@robbiegeoghegan/download-twitter-data-with-10-lines-of-code-42eb2ba1ab0f
#set search variables

oldest_date = datetime.datetime.strptime("2019-09-03", "%Y-%m-%d")
newest_date = datetime.datetime.strptime("2019-12-31", "%Y-%m-%d")
# add in iterator for week or date
date_range = [oldest_date + datetime.timedelta(days = x) for x in range(0,(newest_date-oldest_date).days, 1)]
# https://www.pythonprogramming.in/get-range-of-dates-between-specified-start-and-end-date.html

# tweetCriteria_list = []
# for keyword in query_dow:
#     tweetCriteria = got.manager.TweetCriteria().setQuerySearch(keyword)\
#                                             .setSince(oldest_date)\
#                                             .setUntil(newest_date)
#     tweetCriteria_list.append(tweetCriteria)
#create twitter info for each company



# @sleep_and_retry
# @limits(calls=15, period=60)
def pull_tweets(company):
    
    oldest_date = datetime.datetime.strptime("2019-09-03", "%Y-%m-%d")
    newest_date = datetime.datetime.strptime("2019-12-31", "%Y-%m-%d")
    # add in iterator for week or date
    date_range = [oldest_date + datetime.timedelta(days = x) for x in range(0,(newest_date-oldest_date).days, 1)]
#     for company in dow:
    tweet_list = []
    tweet_dict = {}
    for i, week in enumerate(date_range[:-1]):
        start = str(date_range[i])[:10]
        end = str(date_range[i+1])[:10]
        criteria = got.manager.TweetCriteria().setQuerySearch(f'${company}')\
                                            .setSince(start)\
                                            .setUntil(end)
#                                             .setMaxTweets(3_000)
            
        tweets = got.manager.TweetManager.getTweets(criteria)
        tweet_list += tweets
        if (i+1) % 7 == 0:
            print(f'added week ended {end} for {company}.  {i/len(date_range):.2%}')
            print(f'currently at {len(tweet_list)} tweets')
#                 time.sleep(60)

        
    print(f'pulled {len(tweet_list)} tweets for {company}.')
    #create df
    tweet_dict[company] = tweet_list
    tweet_df = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in tweet_dict.items() ]))
    tweet_df['tweet_count'] = tweet_df.index
    tweet_df = pd.melt(tweet_df, id_vars=["tweet_count"], var_name='company', value_name='got_criteria')
    tweet_df = get_twitter_info(tweet_df)
    tweet_df = tweet_df.drop("got_criteria", 1)
    tweet_df.to_csv(f'./datasets/twitters/{company}_tweets.csv', index = False)
#         time.sleep(180)


# tweet_df = tweet_df.dropna()
#extract twitter information



In [5]:
count = 0
dow = ['dis', 'rtn', 'utx'] 
# 'mmm', 'axp', 'aapl', 'ba', 'cat', 'cvx', 'csco', 'ko', 'dow', 'xom', 'gs', 'hd', 'ibm',
#       'intc', 'jnj', 'jpm', 'mcd', 'mrk', 'msft', 'nke', 'pfe', 
# 'pg', 'trv', 'unh', 'vz', 'v', 'wmt', 'wba', 
for company in dow:
    pull_tweets(company)
    count += 1
    print(f'pulled {count}/{len(dow)}')
    

added week ended 2019-09-10 for dis.  5.04%
currently at 1332 tweets
added week ended 2019-09-17 for dis.  10.92%
currently at 2616 tweets
added week ended 2019-09-24 for dis.  16.81%
currently at 3935 tweets
added week ended 2019-10-01 for dis.  22.69%
currently at 5309 tweets
added week ended 2019-10-08 for dis.  28.57%
currently at 6612 tweets
added week ended 2019-10-15 for dis.  34.45%
currently at 7456 tweets
added week ended 2019-10-22 for dis.  40.34%
currently at 8737 tweets
added week ended 2019-10-29 for dis.  46.22%
currently at 9970 tweets
added week ended 2019-11-05 for dis.  52.10%
currently at 11249 tweets
added week ended 2019-11-12 for dis.  57.98%
currently at 14446 tweets
added week ended 2019-11-19 for dis.  63.87%
currently at 20029 tweets
added week ended 2019-11-26 for dis.  69.75%
currently at 22865 tweets
added week ended 2019-12-03 for dis.  75.63%
currently at 25126 tweets
added week ended 2019-12-10 for dis.  81.51%
currently at 26378 tweets
added week ende

In [11]:
for file in os.listdir('./datasets/twitters/'):
    df = pd.read_csv(f'./datasets/twitters/{file}')
    print(f'{file[:-4]} is {df.shape[0]} long')

cvx_tweets is 4149 long
v_tweets is 6915 long
dow_tweets is 4457 long
cat_tweets is 9469 long
.DS_S is 4 long
mrk_tweets is 6170 long
axp_tweets is 2761 long
jpm_tweets is 12981 long
gs_tweets is 12893 long
rtn_tweets is 2007 long
intc_tweets is 10119 long
xom_tweets is 7911 long
ko_tweets is 4250 long
vz_tweets is 5333 long
nke_tweets is 8843 long
utx_tweets is 2301 long
ibm_tweets is 5801 long
wmt_tweets is 10034 long
mcd_tweets is 10556 long
aapl_tweets is 81967 long
unh_tweets is 5247 long
msft_tweets is 26489 long
ba_tweets is 32647 long
dis_tweets is 30279 long
csco_tweets is 6427 long
wba_tweets is 3438 long
hd_tweets is 7780 long
pg_tweets is 4641 long
trv_tweets is 1321 long
mmm_tweets is 4260 long
jnj_tweets is 8311 long
pfe_tweets is 5873 long


In [10]:
os.listdir('./datasets/twitters/')

['untitled folder',
 'cvx_tweets.csv',
 'v_tweets.csv',
 'dow_tweets.csv',
 'cat_tweets.csv',
 '.DS_Store',
 'mrk_tweets.csv',
 'axp_tweets.csv',
 'jpm_tweets.csv',
 'gs_tweets.csv',
 'rtn_tweets.csv',
 'intc_tweets.csv',
 'xom_tweets.csv',
 'ko_tweets.csv',
 'vz_tweets.csv',
 'nke_tweets.csv',
 'utx_tweets.csv',
 'ibm_tweets.csv',
 'wmt_tweets.csv',
 'mcd_tweets.csv',
 'aapl_tweets.csv',
 'unh_tweets.csv',
 'msft_tweets.csv',
 'ba_tweets.csv',
 'dis_tweets.csv',
 'csco_tweets.csv',
 'wba_tweets.csv',
 'hd_tweets.csv',
 'pg_tweets.csv',
 'trv_tweets.csv',
 'mmm_tweets.csv',
 'jnj_tweets.csv',
 'pfe_tweets.csv']

https://github.com/Mottl/GetOldTweets3/issues/57
https://github.com/Mottl/GetOldTweets3/issues/3#issuecomment-527642499
https://github.com/libbyh/GetOldTweets-python/blob/py3/GetOldTweets3/manager/TweetManager.py#L7
https://github.com/libbyh/GetOldTweets-python/blob/py3/GetOldTweets3/manager/TweetManager.py
https://github.com/tomasbasham/ratelimit
https://github.com/Mottl/GetOldTweets3/blob/master/bin/GetOldTweets3
